In [1]:
import os
import nlp
import time
import torch
import random
import logging

import numpy as np

from typing import List
from typing import Dict, Optional
from dataclasses import dataclass, field
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data.dataset import Dataset, IterableDataset
from transformers import PreTrainedTokenizer, DataCollator, PreTrainedModel
from transformers import AutoTokenizer, EvalPrediction, Trainer, HfArgumentParser, TrainingArguments, \
    AutoModelForSequenceClassification, set_seed, AutoConfig

In [2]:
set_seed(42)

In [3]:
dataset = nlp.load_dataset('glue', 'mnli')

# Let's get an idea of the data format
print(dataset)
print(dataset["train"][0])

Dataset glue downloaded and prepared to /home/mirac13/.cache/huggingface/datasets/glue/mnli/1.0.0/637080968c182118f006d3ea39dd9937940e81cfffc8d79836eaae8bba307fc4. Subsequent calls will reuse this data.
{'train': Dataset(features: {'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradiction'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}, num_rows: 392702), 'validation_matched': Dataset(features: {'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradiction'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}, num_rows: 9815), 'validation_mismatched': Dataset(features: {'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=3, names=['entailment', 'neutral', 'contradict

In [4]:
@dataclass
class Example:
    text_a: str
    text_b: str
    label: int

# to simplify code below, we convert list of dict provided by nlp package to list of Example
train = [Example(text_a=item["premise"], text_b=item["hypothesis"], \
                 label=item["label"]) for item in dataset["train"]]
valid = [Example(text_a=item["premise"], text_b=item["hypothesis"], \
                 label=item["label"]) for item in dataset["validation_matched"]]